#### 1. Get data from web page.

In [1]:
from bs4 import BeautifulSoup
import requests
import re
import os

In [2]:
#参考文章：https://blog.csdn.net/HollyRan/article/details/85254375
#目标网站：https://www.bjsubway.com/station/zjgls/#
#百度百科的站点不太靠谱，从北京地铁官网找到的这个比较好，还包括站点距离

url = 'https://www.bjsubway.com/station/zjgls/#'
#verify必须等于false，否则SSL会报错
response = requests.get(url,verify=False)
#使用中文编码格式gbk进行解码
response.encoding = 'gbk'
html = response.text
#将源代码信息用BeautifulSoup的parser解码器进行解码
soup = BeautifulSoup(html,"html.parser")

C:\ProgramData\Anaconda3\lib\site-packages\urllib3\connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


In [3]:
# 得到线路名称
def get_txt_name():  
    txt_src_name = []
    for i in range(5, 10):
        #观察源代码可知线路信息都在名为"colspan"的字符串后面，且colspan后面的数字在5-10之间
        #使用soup自带的正则表达，字典方式
        temp = soup.find_all('td', {'colspan': str(i)})
        txt_src_name += temp
    return txt_src_name
print(get_txt_name())
#得到准确线路名称，去除边角冗余字符串
def get_txtuseful_name(): 
    obj = []
    for each in get_txt_name():
        # 从>匹配到<(不包含)，用括号包住想要的部分
        temp = re.findall(r">(.+?)<", str(each))  
        obj += temp
    return obj
print('\n')
#查看地铁线路总共有多少
print(get_txtuseful_name())

[<td colspan="5">1号线相邻站间距信息统计表</td>, <td colspan="5">5号线相邻站间距信息统计表</td>, <td colspan="5">6号线相邻站间距信息统计表</td>, <td colspan="5">7号线相邻站间距信息统计表</td>, <td colspan="5">8号线相邻站间距信息统计表</td>, <td colspan="5">9号线相邻站间距信息统计表</td>, <td colspan="5">八通线相邻站间距信息统计表</td>, <td colspan="5">亦庄线相邻站间距信息统计表</td>, <td colspan="5">房山线相邻站间距信息统计表</td>, <td colspan="5">机场线相邻站间距信息统计表</td>, <td colspan="6">15号线相邻站间距信息统计表</td>, <td colspan="6">昌平线相邻站间距信息统计表</td>, <td colspan="7">2号线相邻站间距信息统计表</td>, <td colspan="7">4号线相邻站间距信息统计表</td>, <td colspan="7">13号线相邻站间距信息统计表</td>, <td colspan="7">14号线(西段)相邻站间距信息统计表</td>, <td colspan="7">14号线（东段）相邻站间距信息统计表</td>, <td colspan="7">大兴线相邻站间距信息统计表</td>, <td colspan="9">10号线相邻站间距信息统计表</td>]


['1号线相邻站间距信息统计表', '5号线相邻站间距信息统计表', '6号线相邻站间距信息统计表', '7号线相邻站间距信息统计表', '8号线相邻站间距信息统计表', '9号线相邻站间距信息统计表', '八通线相邻站间距信息统计表', '亦庄线相邻站间距信息统计表', '房山线相邻站间距信息统计表', '机场线相邻站间距信息统计表', '15号线相邻站间距信息统计表', '昌平线相邻站间距信息统计表', '2号线相邻站间距信息统计表', '4号线相邻站间距信息统计表', '13号线相邻站间距信息统计表', '14号线(西段)相邻站间距信息统计表', '14号线（东段）相邻站间距信息统计表'

In [4]:
#分析网页源码，发现所有的站点信息都被存在"<tbody>"下
Station_info = soup.find_all('tbody')
#print(Station_info)
#tbody下的关键信息仍然需要正则提取
def get_station_info():
    obj = []
    for each in Station_info:
        temp = re.findall(r">(.+?)<", str(each))   # 正则匹配，str格式
        obj += temp
    return obj
#print(get_station_info())
#得到站点与位置两个关键信息
station_geo_info = get_station_info()

#### 2. Preprocessing data from page source.

In [5]:
#保存到test文件备用
with open('test.txt','w',encoding='utf-8') as f:
    for line in station_geo_info:
        if line == '上行/下行' or line == '上行' or line == '下行':
            f.write('\n')
        else:
            f.write(line + '   ')

In [6]:
#保存到列表中
info = []
for i in station_geo_info:
    #去除多余的文字
    if i == '上行/下行' or i == '上行' or i == '下行': continue
    else:
        info.append(i)
        
info_1 = {}
n = 0
for i in info:
    n += 1
    #站点的字符串大于5，选出站点形成字典
    #站点作为key，距离作为值，把站点间的字符串‘--’换成‘:’
    if len(i)>5:
        info_1[i.replace('——',':')] = info[n]

In [9]:
#先找出站点间单向连接的关系
station_connection_single = {}
visited = []
for i in info_1.keys():
    #如果某站点在之前出现过，那么此站点还能到达其他点
    #如建国门可到北京站，也可到东单，先设{'建国门':['北京站']},再把东单append进入key对应的value中
    if i.split(':')[0] in visited:
        station_connection_single[i.split(':')[0]].append(i.split(':')[1])
    else:
        station_connection_single[i.split(':')[0]]= [i.split(':')[1]]
        visited.append(i.split(':')[0])
#station_connection_single

In [10]:
#补充前后连接关系
#一个站点除了可以到它的下一个点，它还可以去到它的上一个点
station_connection = station_connection_single
all_station = list(station_connection_single.keys())
c = list(station_connection_single.values())
for i in all_station:
    for p in c:
        #若此站点出现在其他站点可以到达的站点中，则此站点也能到达那个站点
        #如建国门可到站点为[北京站，东单],对于北京站和东单而言，它们也可以到建国门
        #找到建国门的索引号，赋入字典key对应的value中
        if i in p:
            index = c.index(p)
            station_connection[i].append(all_station[index])        
#station_connection     

In [11]:
station_connection['西单'] 

['天安门西', '宣武门', '复兴门', '灵境胡同']

#### 3. Build the search agent

In [12]:
#和课堂代码一样，创建搜索策略
def search(start,stop,connection_graph,sort_candidate):
    pathes = [[start]]
    visited = set()
    
    while pathes:
        path = pathes.pop()
        frontier = path[-1]
        if frontier in visited: continue
        successors = connection_graph[frontier]
        for city in successors:
            if city in path: continue
            new_path = path + [city]
            pathes.append(new_path)
            if city == stop: return new_path
        visited.add(frontier)
        pathes = sort_candidate(pathes)
#最长站点
def transfer_as_much(pathes):
    return sorted(pathes,key=len)
#最少站点
def transfer_as_less(pathes):
    return sorted(pathes,key=len,reverse=True)

In [13]:
#导入emoji
from emoji import emojize
def pretty_print(cities):
    print(emojize(" :metro:-> ").join(cities))

In [14]:
pretty_print(search('建国门','复兴门',station_connection,sort_candidate=transfer_as_less))

建国门 🚇-> 东单 🚇-> 王府井 🚇-> 天安门东 🚇-> 天安门西 🚇-> 西单 🚇-> 复兴门


In [15]:
pretty_print(search('建国门','复兴门',station_connection,sort_candidate=transfer_as_much))

建国门 🚇-> 北京站 🚇-> 崇文门 🚇-> 前门 🚇-> 和平门 🚇-> 宣武门 🚇-> 长椿街 🚇-> 复兴门


In [21]:
pretty_print(search('东单','六道口',station_connection,sort_candidate=transfer_as_less))

东单 🚇-> 灯市口 🚇-> 东四 🚇-> 南锣鼓巷 🚇-> 什刹海 🚇-> 鼓楼大街 🚇-> 安德里北街 🚇-> 安华桥 🚇-> 北土城 🚇-> 奥体中心 🚇-> 奥林匹克公园 🚇-> 北沙滩 🚇-> 六道口
